Stationary Dataset Creation
from the KITTI dataset
In my data folder, there are other folders from the KITTI dataset.
Each of those folders is of a sequence. For each sequence, there is a folder for
- image_00: black and white images from camera 1
- image_01: black and white images from camera 2
- image_02: color images from camera 1
- image_03: color images from camera 2
- oxts: Oxford Technical Solutions - the gps and positional data for the vehicle.
- velodyne_points: the lidar frames from the sequence
- tracklet_labels.xml: the labels for the lidar

What I would like to end up with:
- The data in kitti format. Images, lidar frames, labels, and camera images
(by the way, I would hacve the calibration data for this).

Intermediate data:
- for each sequence folder, I would like to make a folder for each stationary sequence.
- Each one of those folders contains 
    - a set of images, starting from 00000000.
    - a set of corresponding original lidar frames
    - a set of filtered lidar frames
    - (later) a set of labeled points frames
    - corresponding labels for each frame